# L1: Automated Project: Planning, Estimation, and Allocation

<p style="background-color:#fff6e4; padding:15px; border-width:3px; border-color:#f5ecda; border-style:solid; border-radius:6px"> ⏳ <b>Note <code>(Kernel Starting)</code>:</b> This notebook takes about 30 seconds to be ready to use. You may start and watch the video while you wait.</p>

## Initial Imports

In [1]:
__import__('pysqlite3')
import sys
sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')


In [2]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

# Load environment variables
from helper import load_env
load_env()

import os
import yaml
from crewai import Agent, Task, Crew,LLM

<p style="background-color:#fff6ff; padding:15px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px"> 💻 &nbsp; <b>Access <code>requirements.txt</code> and <code>helper.py</code> files:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>. For more help, please see the <em>"Appendix - Tips and Help"</em> Lesson.</p>

## Set OpenAI Model

In [3]:
# os.environ['OPENAI_MODEL_NAME'] = 'gpt-4o-mini'

## Loading Tasks and Agents YAML files

In [4]:
# Define file paths for YAML configurations
files = {
    'agents': 'L1-config/agents.yaml',
    'tasks': 'L1-config/tasks.yaml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
agents_config = configs['agents']
tasks_config = configs['tasks']

In [5]:
# os.environ['OPENAI_MODEL_NAME'] = 'gpt-4o-mini'
INFERENCE_SERVER_URL = "http://localhost:8989"
MODEL_NAME = "ibm-granite/granite-3.3-2b-instruct"
API_KEY= "alanliuxiang"


llm = LLM(model="ibm-granite/granite-3.3-2b-instruct", 
          base_url=f"{INFERENCE_SERVER_URL}/v1",
          api_key=API_KEY,
          custom_llm_provider ="openai",
          max_tokens=1024)

## Create Pydantic Models for Structured Output

In [6]:
from typing import List
from pydantic import BaseModel, Field

class TaskEstimate(BaseModel):
    task_name: str = Field(..., description="Name of the task")
    estimated_time_hours: float = Field(..., description="Estimated time to complete the task in hours")
    required_resources: List[str] = Field(..., description="List of resources required to complete the task")

class Milestone(BaseModel):
    milestone_name: str = Field(..., description="Name of the milestone")
    tasks: List[str] = Field(..., description="List of task IDs associated with this milestone")

class ProjectPlan(BaseModel):
    tasks: List[TaskEstimate] = Field(..., description="List of tasks with their estimates")
    milestones: List[Milestone] = Field(..., description="List of project milestones")

## Create Crew, Agents and Tasks

In [7]:
# Creating Agents
project_planning_agent = Agent(
  config=agents_config['project_planning_agent'],
  llm=llm,
)

estimation_agent = Agent(
  config=agents_config['estimation_agent'],
  llm=llm,
)

resource_allocation_agent = Agent(
  config=agents_config['resource_allocation_agent'],
  llm=llm,
)

# Creating Tasks
task_breakdown = Task(
  config=tasks_config['task_breakdown'],
  agent=project_planning_agent
)

time_resource_estimation = Task(
  config=tasks_config['time_resource_estimation'],
  agent=estimation_agent
)

resource_allocation = Task(
  config=tasks_config['resource_allocation'],
  agent=resource_allocation_agent,
  output_pydantic=ProjectPlan # This is the structured output we want
)

# Creating Crew
crew = Crew(
  agents=[
    project_planning_agent,
    estimation_agent,
    resource_allocation_agent
  ],
  tasks=[
    task_breakdown,
    time_resource_estimation,
    resource_allocation
  ],
  verbose=True
)

## Crew's Inputs

In [8]:
from IPython.display import display, Markdown

project = 'Website'
industry = 'Technology'
project_objectives = 'Create a website for a small business'
team_members = """
- John Doe (Project Manager)
- Jane Doe (Software Engineer)
- Bob Smith (Designer)
- Alice Johnson (QA Engineer)
- Tom Brown (QA Engineer)
"""
project_requirements = """
- Create a responsive design that works well on desktop and mobile devices
- Implement a modern, visually appealing user interface with a clean look
- Develop a user-friendly navigation system with intuitive menu structure
- Include an "About Us" page highlighting the company's history and values
- Design a "Services" page showcasing the business's offerings with descriptions
- Create a "Contact Us" page with a form and integrated map for communication
- Implement a blog section for sharing industry news and company updates
- Ensure fast loading times and optimize for search engines (SEO)
- Integrate social media links and sharing capabilities
- Include a testimonials section to showcase customer feedback and build trust
"""

# Format the dictionary as Markdown for a better display in Jupyter Lab
formatted_output = f"""
**Project Type:** {project}

**Project Objectives:** {project_objectives}

**Industry:** {industry}

**Team Members:**
{team_members}
**Project Requirements:**
{project_requirements}
"""
# Display the formatted output as Markdown
display(Markdown(formatted_output))


**Project Type:** Website

**Project Objectives:** Create a website for a small business

**Industry:** Technology

**Team Members:**

- John Doe (Project Manager)
- Jane Doe (Software Engineer)
- Bob Smith (Designer)
- Alice Johnson (QA Engineer)
- Tom Brown (QA Engineer)

**Project Requirements:**

- Create a responsive design that works well on desktop and mobile devices
- Implement a modern, visually appealing user interface with a clean look
- Develop a user-friendly navigation system with intuitive menu structure
- Include an "About Us" page highlighting the company's history and values
- Design a "Services" page showcasing the business's offerings with descriptions
- Create a "Contact Us" page with a form and integrated map for communication
- Implement a blog section for sharing industry news and company updates
- Ensure fast loading times and optimize for search engines (SEO)
- Integrate social media links and sharing capabilities
- Include a testimonials section to showcase customer feedback and build trust



## Kicking off the crew

In [9]:
# The given Python dictionary
inputs = {
  'project_type': project,
  'project_objectives': project_objectives,
  'industry': industry,
  'team_members': team_members,
  'project_requirements': project_requirements
}

# Run the crew
result = crew.kickoff(
  inputs=inputs
)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: e9c8beae-c245-4a14-be32-2e5a2e9e0cd8                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Provider List: https://docs.litellm.ai/docs/providers

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: The Ultimate Project Planner                                                                            │
│                                                                                                                 │
│  Task: Carefully analyze the project_requirements for the Website project and break them down into individual   │
│  tasks. Define each task's scope in detail, set achievable timelines, and ensure that all dependencies are      │
│  accounted for:                                                                                                 │
│                                                                                                                 │
│  - Create a responsive design that works well on desktop and mobile devices                                     │
│  - Implement a modern, visually appealing user interface with a clean look                                      │
│  - Develop a user-friendly navigation system with intuitive menu structure                                      │
│  - Include an "About Us" page highlighting the company's history and values                                     │
│  - Design a "Services" page showcasing the business's offerings with descriptions                               │
│  - Create a "Contact Us" page with a form and integrated map for communication                                  │
│  - Implement a blog section for sharing industry news and company updates                                       │
│  - Ensure fast loading times and optimize for search engines (SEO)                                              │
│  - Integrate social media links and sharing capabilities                                                        │
│  - Include a testimonials section to showcase customer feedback and build trust                                 │
│                                                                                                                 │
│                                                                                                                 │
│  Team members:                                                                                                  │
│                                                                                                                 │
│  - John Doe (Project Manager)                                                                                   │
│  - Jane Doe (Software Engineer)                                                                                 │
│  - Bob Smith (Designer)                                                                                         │
│  - Alice Johnson (QA Engineer)                                                                                  │
│  - Tom Brown (QA Engineer)                                                                                      │
│                                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: The Ultimate Project Planner                                                                            │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Here’s a detailed task breakdown for the Website project, ensuring each aspect is meticulously addressed,      │
│  including dependencies, timelines, and deliverables for later creation of a Gantt chart:                       │
│                                                                                                                 │
│  ### Task Breakdown                                                                                             │
│                                                                                                                 │
│  1. **Project Kickoff & Requirements Analysis**                                                                 │
│     - **Team Member**: John Doe                                                                                 │
│     - **Description**: Cause a kick-off meeting and analyze the current project requirements in detail with     │
│  stakeholders. Capture high-level requirements, project goals, and any specific technical specifications.       │
│     - **Timeline**: Week 1                                                                                      │
│     - **Dependencies**: None                                                                                    │
│     - **Deliverables**:                                                                                         │
│       - Detailed logic model                                                                                    │
│       - Finalized project requirements document                                                                 │
│                                                                                                                 │
│  2. **Design Development**                                                                                      │
│     - **Team Members**: Bob Smith, Jane Doe                                                                     │
│     - **Description**: Develop a responsive website design that is visually appealing and clean. Create         │
│  mockups and wireframes for both desktop and mobile.                                                            │
│     - **Timeline**: Week 2-3                                                                                    │
│     - **Dependencies**: Completion of project requirements review                                               │
│     - **Deliverables**:                                                                                         │
│       - Responsive design mockups                                                                               │
│       - User interface design (UI)                                                                              │
│       - Design sign-off from stakeholders                                                                       │
│                                                                                                                 │
│  3. **User Interface (UI) Implementation**                                                                      │
│     - **Team Member**: Jane Doe                                                                                 │
│     - **Description**: Based on the design, implement the UI elements to structurally and visually complete     │
│  the platform.                                         

...

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Resource Allocation Strategist                                                                          │
│                                                                                                                 │
│  Task: Strategically allocate tasks for the Website project to team members based on their skills,              │
│  availability, and current workload. Ensure that each task is assigned to the most suitable team member and     │
│  that the workload is evenly distributed.                                                                       │
│                                                                                                                 │
│  Team members:                                                                                                  │
│                                                                                                                 │
│  - John Doe (Project Manager)                                                                                   │
│  - Jane Doe (Software Engineer)                                                                                 │
│  - Bob Smith (Designer)                                                                                         │
│  - Alice Johnson (QA Engineer)                                                                                  │
│  - Tom Brown (QA Engineer)                                                                                      │
│                                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Resource Allocation Strategist                                                                          │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  {                                                                                                              │
│    "tasks": [                                                                                                   │
│      {                                                                                                          │
│        "task_name": "Project Kickoff & Requirements Analysis",                                                  │
│        "estimated_time_hours": 16.0,                                                                            │
│        "required_resources": ["John Doe"]                                                                       │
│      },                                                                                                         │
│      {                                                                                                          │
│        "task_name": "Design Development",                                                                       │
│        "estimated_time_hours": 24.0,                                                                            │
│        "required_resources": ["Bob Smith", "Jane Doe"]                                                          │
│      },                                                                                                         │
│      {                                                                                                          │
│        "task_name": "User Interface (UI) Implementation",                                                       │
│        "estimated_time_hours": 32.0,                                                                            │
│        "required_resources": ["Jane Doe"]                                                                       │
│      },                                                                                                         │
│      {                                                                                                          │
│        "task_name": "Navigation Structure Development",                                                         │
│        "estimated_time_hours": 8.0,                                                                             │
│        "required_resources": ["Jane Doe"]                                                                       │
│      },                                                                                                         │
│      {                                                                                                          │
│        "task_name": "Content Creation - About Us & Services",                                                   │
│        "estimated_time_hours": 16.0,                                                                            │
│        "required_resources": ["Bob Smith", "(Content Generation Support)"]                                      │
│      },                                                                                                         │
│      {                                                                                                          │
│        "task_name": "Contact & Blog Implementation",                                                            │
│        "estimated_time_hours": 24.0,                   

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 346f15fa-6efb-4705-b82a-7a442f5ca19c                                                                     │
│  Agent: Resource Allocation Strategist                                                                          │
│                                                                                                                 │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: e9c8beae-c245-4a14-be32-2e5a2e9e0cd8                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: {                                                                                                │
│    "tasks": [                                                                                                   │
│      {                                                                                                          │
│        "task_name": "Project Kickoff & Requirements Analysis",                                                  │
│        "estimated_time_hours": 16.0,                                                                            │
│        "required_resources": ["John Doe"]                                                                       │
│      },                                                                                                         │
│      {                                                                                                          │
│        "task_name": "Design Development",                                                                       │
│        "estimated_time_hours": 24.0,                                                                            │
│        "required_resources": ["Bob Smith", "Jane Doe"]                                                          │
│      },                                                                                                         │
│      {                                                                                                          │
│        "task_name": "User Interface (UI) Implementation",                                                       │
│        "estimated_time_hours": 32.0,                                                                            │
│        "required_resources": ["Jane Doe"]                                                                       │
│      },                                                                                                         │
│      {                                                                                                          │
│        "task_name": "Navigation Structure Development",                                                         │
│        "estimated_time_hours": 8.0,                                                                             │
│        "required_resources": ["Jane Doe"]                                                                       │
│      },                                                                                                         │
│      {                                                                                                          │
│        "task_name": "Content Creation - About Us & Services",                                                   │
│        "estimated_time_hours": 16.0,                                                                            │
│        "required_resources": ["Bob Smith", "(Content Generation Support)"]                                      │
│      },                                                                                                         │
│      {                                                                                                          │
│        "task_name": "Contact & Blog Implementation",  

## Usage Metrics and Costs

Let’s see how much it would cost each time if this crew runs at scale.

In [10]:
import pandas as pd

costs = 0.150 * (crew.usage_metrics.prompt_tokens + crew.usage_metrics.completion_tokens) / 1_000_000
print(f"Total costs: ${costs:.4f}")

# Convert UsageMetrics instance to a DataFrame
df_usage_metrics = pd.DataFrame([crew.usage_metrics.dict()])
df_usage_metrics

Total costs: $0.0010


,total_tokens,prompt_tokens,cached_prompt_tokens,completion_tokens,successful_requests
0,6979,4300,0,2679,3


## Result

In [11]:
result.pydantic.dict()

{'tasks': [{'task_name': 'Project Kickoff & Requirements Analysis',
   'estimated_time_hours': 16.0,
   'required_resources': ['John Doe']},
  {'task_name': 'Design Development',
   'estimated_time_hours': 24.0,
   'required_resources': ['Bob Smith', 'Jane Doe']},
  {'task_name': 'User Interface (UI) Implementation',
   'estimated_time_hours': 32.0,
   'required_resources': ['Jane Doe']},
  {'task_name': 'Navigation Structure Development',
   'estimated_time_hours': 8.0,
   'required_resources': ['Jane Doe']},
  {'task_name': 'Content Creation - About Us & Services',
   'estimated_time_hours': 16.0,
   'required_resources': ['Bob Smith', '(Content Generation Support)']},
  {'task_name': 'Contact & Blog Implementation',
   'estimated_time_hours': 24.0,
   'required_resources': ['Jane Doe', '(Web Developer Support)']},
  {'task_name': 'SEO & Performance Optimization',
   'estimated_time_hours': 24.0,
   'required_resources': ['John Doe']},
  {'task_name': 'Social Media & Feedback Section

## Inspect further

In [12]:
tasks = result.pydantic.dict()['tasks']
df_tasks = pd.DataFrame(tasks)

# Display the DataFrame as an HTML table
df_tasks.style.set_table_attributes('border="1"').set_caption("Task Details").set_table_styles(
    [{'selector': 'th, td', 'props': [('font-size', '120%')]}]
)

,task_name,estimated_time_hours,required_resources
0,Project Kickoff & Requirements Analysis,16.000000,['John Doe']
1,Design Development,24.000000,"['Bob Smith', 'Jane Doe']"
2,User Interface (UI) Implementation,32.000000,['Jane Doe']
3,Navigation Structure Development,8.000000,['Jane Doe']
4,Content Creation - About Us & Services,16.000000,"['Bob Smith', '(Content Generation Support)']"
5,Contact & Blog Implementation,24.000000,"['Jane Doe', '(Web Developer Support)']"
6,SEO & Performance Optimization,24.000000,['John Doe']
7,Social Media & Feedback Section Integration,16.000000,"['Alice Johnson', 'Bob Smith']"
8,Quality Assurance Testing,32.000000,"['Alice Johnson', 'Tom Brown']"
9,Deployment & Studio Handover,16.000000,['John Doe']


### Inspecting Milestones

In [13]:
milestones = result.pydantic.dict()['milestones']
df_milestones = pd.DataFrame(milestones)

# Display the DataFrame as an HTML table
df_milestones.style.set_table_attributes('border="1"').set_caption("Task Details").set_table_styles(
    [{'selector': 'th, td', 'props': [('font-size', '120%')]}]
)

,milestone_name,tasks
0,Phase 1 Completion,"['Project Kickoff & Requirements Analysis', 'Design Development', 'Navigation Structure Development', 'Content Creation - About Us & Services']"
1,Phase 2 Completion,"['User Interface (UI) Implementation', 'Contact & Blog Implementation', 'SEO & Performance Optimization']"
2,Phase 3 Completion,"['Social Media & Feedback Section Integration', 'Quality Assurance Testing']"
3,Final Handover & Deployment,['Deployment & Studio Handover']
